# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [97]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [98]:
# your code here
df_csv = pd.read_csv('./../data/Admission_Predict.csv')


Remove trailing spaces at the end of the column names if there are any.


In [99]:
# your code here
df_csv.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [100]:
df_csv.columns = df_csv.columns.str.strip()
df_csv.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [101]:
# your code here
df_csv.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [102]:
# your code here
df_csv.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [103]:
# your code here
df_csv.set_index('Serial No.', drop= False)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [104]:
# your code here
df_csv.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [105]:
df_csv['GRE Score - CGPA']= df_csv['GRE Score'].astype(str)+'-'+ df_csv['CGPA'].astype(str)
if df_csv['GRE Score - CGPA'].nunique()== len(df_csv):
    print ('it is unique!')

it is unique!


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [106]:
df_csv.set_index(['GRE Score', 'CGPA'], inplace=True)



Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [107]:
# your code here
df_csv.reset_index(inplace=True)
df_csv.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE Score - CGPA
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [108]:
# your code here
len(df_csv.query("CGPA > 9 & Research==1"))



101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [109]:
# your code here
df_filtered= df_csv.query("CGPA > 9 & SOP < 3.5")
df_filtered ['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [115]:
def standardize(df, col):
    name= col+ '_std'
    df[col+ '_std']=(df[col]-df[col].mean())/df[col].std()
    return df
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """

    
df_csv.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE Score - CGPA,CGPA_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65,1.747900
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0,-0.991211
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67,0.121034
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21,-0.642597
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34,1.233279


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [119]:
# your code here
standardize(df_csv,"CGPA" )
standardize(df_csv,"GRE Score" )
standardize(df_csv,"LOR" )
admissions = df_csv.copy()
admissions.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE Score - CGPA,CGPA_std,GRE Score_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65,1.747900,1.753381,1.191647
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0,-0.991211,-0.063367,0.076740
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67,0.121034,0.455704,-1.038166
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21,-0.642597,-0.236391,-0.480713
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [120]:
# Libraries
from random import choices

In [123]:
std_columns = ['CGPA_std', 'GRE Score_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
decision_choice

['CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE Score_std',
 'LOR_std',
 'GRE Score_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE Score_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE Score_std',
 'GRE Score_std',
 'CGPA_std',
 'LOR_std',
 'GRE Score_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE Score_std',
 'GRE Score_std',
 'LOR_std',
 'GRE Score_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE Score_std',
 'CGPA_std',
 'LOR_std',
 'GRE Score_std',
 'GRE Score_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE Score_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE Score_std',
 'GRE Score_std',
 'GRE Score_std',
 'LOR_std',
 'GRE Score_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GR

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [132]:
# your code here
admissions['deciding_column']= admissions.lookup(admissions.index, decision_choice)
admissions.head()


/Users/Izaskun/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  


,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE Score - CGPA,CGPA_std,GRE Score_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65,1.747900,1.753381,1.191647,1.747900
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0,-0.991211,-0.063367,0.076740,-0.991211
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67,0.121034,0.455704,-1.038166,0.121034
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21,-0.642597,-0.236391,-0.480713,-0.642597
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34,1.233279,1.147798,-0.480713,1.233279


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [145]:
# your code here
admissions['decision']= admissions.eval( 'deciding_column>0.8' ) 
admissions.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE Score - CGPA,CGPA_std,GRE Score_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65,1.747900,1.753381,1.191647,1.747900,True
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0,-0.991211,-0.063367,0.076740,-0.991211,False
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67,0.121034,0.455704,-1.038166,0.121034,False
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21,-0.642597,-0.236391,-0.480713,-0.642597,False
4,330,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34,1.233279,1.147798,-0.480713,1.233279,True


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [153]:
# your code here
admissions['decision'].value_counts()




False    295
True      90
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [140]:
# your code here
admissions.columns

Index(['GRE Score', 'CGPA', 'Serial No.', 'TOEFL Score', 'University Rating',
       'SOP', 'LOR', 'Research', 'Chance of Admit', 'GRE Score - CGPA',
       'CGPA_std', 'GRE Score_std', 'LOR_std', 'deciding_column'],
      dtype='object')

In [155]:
admissions.columns=admissions.columns.str.lower().str.replace(' ','').str.replace('.','').str.replace('-','').str.replace('_','')
admissions.columns  

/Users/Izaskun/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


Index(['grescore', 'cgpa', 'serialno', 'toeflscore', 'universityrating', 'sop',
       'lor', 'research', 'chanceofadmit', 'grescorecgpa', 'cgpastd',
       'grescorestd', 'lorstd', 'decidingcolumn', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [164]:
# your code here
admissions.head()
admissions['extra GRE score']= admissions.eval( 'universityrating> 4' ) 
admissions.loc[admissions['universityrating'] > 4, 'extra GRE score'] = 10
admissions.loc[admissions['universityrating'] <= 4, 'extra GRE score'] = 0
admissions ['adjusted_gre']= admissions ['grescore']+ admissions ['extra GRE score']

admissions.head()

,grescore,cgpa,serialno,toeflscore,universityrating,sop,lor,research,chanceofadmit,grescorecgpa,cgpastd,grescorestd,lorstd,decidingcolumn,decision,extra GRE score,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,337-9.65,1.747900,1.753381,1.191647,1.747900,True,0,337
1,316,8.00,2,104,3,3.0,3.5,1,0.72,316-8.0,-0.991211,-0.063367,0.076740,-0.991211,False,0,316
2,322,8.67,3,110,3,3.5,2.5,1,0.80,322-8.67,0.121034,0.455704,-1.038166,0.121034,False,0,322
3,314,8.21,4,103,2,2.0,3.0,0,0.65,314-8.21,-0.642597,-0.236391,-0.480713,-0.642597,False,0,314
4,340,9.34,5,115,5,4.5,3.0,1,0.90,330-9.34,1.233279,1.147798,-0.480713,1.233279,True,10,350


In [166]:
pivot_table = admissions.pivot_table(index=['grescorecgpa'])

In [168]:
admissions['adjusted_gre_bin'] = pivot_table.cut(admissions['adjusted_gre'], 3)

AttributeError: 'DataFrame' object has no attribute 'cut'

In [170]:
pivot_table['adjusted_gre']

KeyError: 'adjusted_gre'